In [48]:
from pyspark.sql.types import *

In [2]:
!wget https://github.com/TobiasGlaubach/python-ml-turbofan/archive/master.zip

--2020-01-23 10:42:06--  https://github.com/TobiasGlaubach/python-ml-turbofan/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/TobiasGlaubach/python-ml-turbofan/zip/master [following]
--2020-01-23 10:42:07--  https://codeload.github.com/TobiasGlaubach/python-ml-turbofan/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'master.zip'

    [                 <=>                   ] 38,999,196  5.26MB/s   in 7.1s   

2020-01-23 10:42:15 (5.26 MB/s) - 'master.zip' saved [38999196]



In [3]:
!unzip master.zip


Archive:  master.zip
2a96a8034f6442e6daac536222c1fc2202274cf7
   creating: python-ml-turbofan-master/
  inflating: python-ml-turbofan-master/.gitignore  
   creating: python-ml-turbofan-master/CMAPSSData/
  inflating: python-ml-turbofan-master/CMAPSSData/Damage Propagation Modeling.pdf  
  inflating: python-ml-turbofan-master/CMAPSSData/RUL_FD001.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/RUL_FD002.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/RUL_FD003.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/RUL_FD004.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/final_test.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/readme.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/test_FD001.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/test_FD002.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/test_FD003.txt  
  inflating: python-ml-turbofan-master/CMAPSSData/test_FD004.txt  
  inflating: python-ml-turbofan-master/CMAPSSData

In [4]:
!pwd

/home/spark/shared


In [5]:
!cp Predictive-Maintenance-In-PySpark-master.zip  predictive

cp: cannot stat 'Predictive-Maintenance-In-PySpark-master.zip': No such file or directory


In [6]:
path='/home/spark/shared/python-ml-turbofan-master/CMAPSSData'
print(path)


/home/spark/shared/python-ml-turbofan-master/CMAPSSData


In [7]:
os.listdir(path)

['Damage Propagation Modeling.pdf',
 'RUL_FD001.txt',
 'RUL_FD002.txt',
 'RUL_FD003.txt',
 'RUL_FD004.txt',
 'final_test.txt',
 'readme.txt',
 'test_FD001.txt',
 'test_FD002.txt',
 'test_FD003.txt',
 'test_FD004.txt',
 'train_FD001.txt',
 'train_FD002.txt',
 'train_FD003.txt',
 'train_FD004.txt']

In [8]:
train_files= [filename for filename in os.listdir(path) if(filename.startswith('train') and filename.endswith('.txt'))]
                                                           

In [9]:
train_files[0]

'train_FD001.txt'

In [10]:
test_files= [filename for filename in os.listdir(path) if(filename.startswith('test') and filename.endswith('.txt'))]

In [11]:
print(train_files)
test_files

['train_FD001.txt', 'train_FD002.txt', 'train_FD003.txt', 'train_FD004.txt']


['test_FD001.txt', 'test_FD002.txt', 'test_FD003.txt', 'test_FD004.txt']

In [12]:
!ls

conda  logs  master.zip  python-ml-turbofan-master  spark-events  user-libs


In [13]:
train_rdd=sc.textFile(path+'/'+train_files[0])

In [14]:
train_rdd=train_rdd.map(lambda line:line.split(" "))

In [15]:
train_rdd.count()

20631

In [16]:
test_rdd=sc.textFile(path+'/'+test_files[0])
test_rdd=test_rdd.map(lambda line:line.split(" "))

In [17]:
test_rdd.count()

13096

In [18]:
header = ["id", "cycle", "setting1", "setting2", "setting3",
                    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8",
                    "s9", "s10", "s11", "s12", "s13", "s14", "s15", 
                    "s16","s17", "s18", "s19", "s20", "s21"]

In [19]:
train_df=train_rdd.toDF(header)

In [20]:
train_df.collect()

[Row(id='1', cycle='1', setting1='-0.0007', setting2='-0.0004', setting3='100.0', s1='518.67', s2='641.82', s3='1589.70', s4='1400.60', s5='14.62', s6='21.61', s7='554.36', s8='2388.06', s9='9046.19', s10='1.30', s11='47.47', s12='521.66', s13='2388.02', s14='8138.62', s15='8.4195', s16='0.03', s17='392', s18='2388', s19='100.00', s20='39.06', s21='23.4190', _27='', _28=''),
 Row(id='1', cycle='2', setting1='0.0019', setting2='-0.0003', setting3='100.0', s1='518.67', s2='642.15', s3='1591.82', s4='1403.14', s5='14.62', s6='21.61', s7='553.75', s8='2388.04', s9='9044.07', s10='1.30', s11='47.49', s12='522.28', s13='2388.07', s14='8131.49', s15='8.4318', s16='0.03', s17='392', s18='2388', s19='100.00', s20='39.00', s21='23.4236', _27='', _28=''),
 Row(id='1', cycle='3', setting1='-0.0043', setting2='0.0003', setting3='100.0', s1='518.67', s2='642.35', s3='1587.99', s4='1404.20', s5='14.62', s6='21.61', s7='554.26', s8='2388.08', s9='9052.94', s10='1.30', s11='47.27', s12='522.42', s13='2

In [21]:
test_df=test_rdd.toDF(header)

In [28]:
test_df.show()

+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+----+-----+------+-------+-------+------+----+---+----+------+-----+-------+---+---+
| id|cycle|setting1|setting2|setting3|    s1|    s2|     s3|     s4|   s5|   s6|    s7|     s8|     s9| s10|  s11|   s12|    s13|    s14|   s15| s16|s17| s18|   s19|  s20|    s21|_27|_28|
+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+----+-----+------+-------+-------+------+----+---+----+------+-----+-------+---+---+
|  1|    1|  0.0023|  0.0003|   100.0|518.67|643.02|1585.29|1398.21|14.62|21.61|553.90|2388.04|9050.17|1.30|47.20|521.72|2388.03|8125.55|8.4052|0.03|392|2388|100.00|38.86|23.3735|   |   |
|  1|    2| -0.0027| -0.0003|   100.0|518.67|641.71|1588.45|1395.42|14.62|21.61|554.85|2388.01|9054.42|1.30|47.50|522.16|2388.06|8139.62|8.3803|0.03|393|2388|100.00|39.02|23.3916|   |   |
|  1|    3|  0.0003|  0.0001|   100.0|518.67|642.46|1586.94|

In [22]:
import pandas as pd
train_pandas_df=train_df.toPandas()

In [23]:
train_pandas_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,_27,_28
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.00,39.06,23.4190,,
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.00,39.00,23.4236,,
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.00,38.95,23.3442,,
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.00,38.88,23.3739,,
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.00,38.90,23.4044,,


In [25]:
test_pandas_df=test_df.toPandas()
test_pandas_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,_27,_28
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.00,38.86,23.3735,,
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.00,39.02,23.3916,,
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.00,39.08,23.4166,,
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.00,39.00,23.3737,,
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.00,38.99,23.4130,,


In [32]:
train_pandas_df.isnull().sum()


id          0
cycle       0
setting1    0
setting2    0
setting3    0
s1          0
s2          0
s3          0
s4          0
s5          0
s6          0
s7          0
s8          0
s9          0
s10         0
s11         0
s12         0
s13         0
s14         0
s15         0
s16         0
s17         0
s18         0
s19         0
s20         0
s21         0
_27         0
_28         0
dtype: int64

In [39]:
train_df.na.drop()

DataFrame[id: string, cycle: string, setting1: string, setting2: string, setting3: string, s1: string, s2: string, s3: string, s4: string, s5: string, s6: string, s7: string, s8: string, s9: string, s10: string, s11: string, s12: string, s13: string, s14: string, s15: string, s16: string, s17: string, s18: string, s19: string, s20: string, s21: string, _27: string, _28: string]

In [40]:
test_df.na.drop()

DataFrame[id: string, cycle: string, setting1: string, setting2: string, setting3: string, s1: string, s2: string, s3: string, s4: string, s5: string, s6: string, s7: string, s8: string, s9: string, s10: string, s11: string, s12: string, s13: string, s14: string, s15: string, s16: string, s17: string, s18: string, s19: string, s20: string, s21: string, _27: string, _28: string]

In [42]:
train_pandas_df.dropna()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,_27,_28
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.00,39.06,23.4190,,
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.00,39.00,23.4236,,
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.00,38.95,23.3442,,
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.00,38.88,23.3739,,
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.00,38.90,23.4044,,
5,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,8132.85,8.4108,0.03,391,2388,100.00,38.98,23.3669,,
6,1,7,0.0010,0.0001,100.0,518.67,642.48,1592.32,1397.77,14.62,...,8132.32,8.3974,0.03,392,2388,100.00,39.10,23.3774,,
7,1,8,-0.0034,0.0003,100.0,518.67,642.56,1582.96,1400.97,14.62,...,8131.07,8.4076,0.03,391,2388,100.00,38.97,23.3106,,
8,1,9,0.0008,0.0001,100.0,518.67,642.12,1590.98,1394.80,14.62,...,8125.69,8.3728,0.03,392,2388,100.00,39.05,23.4066,,
9,1,10,-0.0033,0.0001,100.0,518.67,641.71,1591.24,1400.46,14.62,...,8129.38,8.4286,0.03,393,2388,100.00,38.95,23.4694,,


In [49]:
df = train_df.withColumn('id', train_df['id'].cast(IntegerType()))

In [52]:
df.collect()

[Row(id=1, cycle='1', setting1='-0.0007', setting2='-0.0004', setting3='100.0', s1='518.67', s2='641.82', s3='1589.70', s4='1400.60', s5='14.62', s6='21.61', s7='554.36', s8='2388.06', s9='9046.19', s10='1.30', s11='47.47', s12='521.66', s13='2388.02', s14='8138.62', s15='8.4195', s16='0.03', s17='392', s18='2388', s19='100.00', s20='39.06', s21='23.4190', _27='', _28=''),
 Row(id=1, cycle='2', setting1='0.0019', setting2='-0.0003', setting3='100.0', s1='518.67', s2='642.15', s3='1591.82', s4='1403.14', s5='14.62', s6='21.61', s7='553.75', s8='2388.04', s9='9044.07', s10='1.30', s11='47.49', s12='522.28', s13='2388.07', s14='8131.49', s15='8.4318', s16='0.03', s17='392', s18='2388', s19='100.00', s20='39.00', s21='23.4236', _27='', _28=''),
 Row(id=1, cycle='3', setting1='-0.0043', setting2='0.0003', setting3='100.0', s1='518.67', s2='642.35', s3='1587.99', s4='1404.20', s5='14.62', s6='21.61', s7='554.26', s8='2388.08', s9='9052.94', s10='1.30', s11='47.27', s12='522.42', s13='2388.03

In [70]:
def set_type(df,intlist=[],doublelist=[]):
    if len(intlist)>0:
        for col in intlist:
            df=df.withColumn(col,df[col].cast(IntegerType()))
                             
    if len(doublelist)>0:
        for col in doublelist:
            df=df.withColumn(col,df[col].cast(DoubleType()))
    return df                            
                             

In [65]:
train_df.columns

['id',
 'cycle',
 'setting1',
 'setting2',
 'setting3',
 's1',
 's2',
 's3',
 's4',
 's5',
 's6',
 's7',
 's8',
 's9',
 's10',
 's11',
 's12',
 's13',
 's14',
 's15',
 's16',
 's17',
 's18',
 's19',
 's20',
 's21',
 '_27',
 '_28']

In [68]:
int_list = ["id", "cycle"]
double_list= list(set(train_df.columns)-set(int_list) )
double_list

['s14',
 's10',
 's16',
 '_27',
 's5',
 's7',
 's21',
 's20',
 's15',
 's11',
 's9',
 's1',
 's17',
 's6',
 's8',
 '_28',
 'setting2',
 'setting1',
 'setting3',
 's13',
 's18',
 's12',
 's4',
 's2',
 's19',
 's3']

In [71]:
train_df=set_type(train_df,int_list,double_list)
test_df=set_type(test_df,int_list,double_list)

In [72]:
train_df.columns

['id',
 'cycle',
 'setting1',
 'setting2',
 'setting3',
 's1',
 's2',
 's3',
 's4',
 's5',
 's6',
 's7',
 's8',
 's9',
 's10',
 's11',
 's12',
 's13',
 's14',
 's15',
 's16',
 's17',
 's18',
 's19',
 's20',
 's21',
 '_27',
 '_28']

In [75]:
test_df.schema

StructType(List(StructField(id,IntegerType,true),StructField(cycle,IntegerType,true),StructField(setting1,DoubleType,true),StructField(setting2,DoubleType,true),StructField(setting3,DoubleType,true),StructField(s1,DoubleType,true),StructField(s2,DoubleType,true),StructField(s3,DoubleType,true),StructField(s4,DoubleType,true),StructField(s5,DoubleType,true),StructField(s6,DoubleType,true),StructField(s7,DoubleType,true),StructField(s8,DoubleType,true),StructField(s9,DoubleType,true),StructField(s10,DoubleType,true),StructField(s11,DoubleType,true),StructField(s12,DoubleType,true),StructField(s13,DoubleType,true),StructField(s14,DoubleType,true),StructField(s15,DoubleType,true),StructField(s16,DoubleType,true),StructField(s17,DoubleType,true),StructField(s18,DoubleType,true),StructField(s19,DoubleType,true),StructField(s20,DoubleType,true),StructField(s21,DoubleType,true),StructField(_27,DoubleType,true),StructField(_28,DoubleType,true)))

In [76]:
train_df.schema

StructType(List(StructField(id,IntegerType,true),StructField(cycle,IntegerType,true),StructField(setting1,DoubleType,true),StructField(setting2,DoubleType,true),StructField(setting3,DoubleType,true),StructField(s1,DoubleType,true),StructField(s2,DoubleType,true),StructField(s3,DoubleType,true),StructField(s4,DoubleType,true),StructField(s5,DoubleType,true),StructField(s6,DoubleType,true),StructField(s7,DoubleType,true),StructField(s8,DoubleType,true),StructField(s9,DoubleType,true),StructField(s10,DoubleType,true),StructField(s11,DoubleType,true),StructField(s12,DoubleType,true),StructField(s13,DoubleType,true),StructField(s14,DoubleType,true),StructField(s15,DoubleType,true),StructField(s16,DoubleType,true),StructField(s17,DoubleType,true),StructField(s18,DoubleType,true),StructField(s19,DoubleType,true),StructField(s20,DoubleType,true),StructField(s21,DoubleType,true),StructField(_27,DoubleType,true),StructField(_28,DoubleType,true)))